# HexMagic

> Fill in a module description here

```python
#| default_exp core
```

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

How do I generate local documentation with nbdev

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To generate local documentation with nbdev, use:

```bash
nbdev_docs
```

This will build the documentation site locally in the `_docs` directory. You can then preview it by opening `_docs/index.html` in your browser, or by serving it with a local web server:

```bash
python -m http.server --directory _docs
```

If you want the docs to automatically rebuild when you make changes, you can use:

```bash
nbdev_preview
```

This starts a local server (typically at `http://localhost:3000`) and watches for changes to your notebooks, rebuilding the docs automatically.

In [ ]:
#| export
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.primitives import Hex, MapCord 


from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import HexGrid, HexPosition ,  HexRegion, GosperCurve

import numpy as np

from HexMagic.terrain import Terrain
from HexMagic.voronoi import generate_plate_terrain

Terrain.fromSeeds = generate_plate_terrain



In [ ]:
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()

In [ ]:
demoTerr()

Initial seeds: 16 (spacing=8)
After elimination: 9
Final unique seeds: 9


In [ ]:
from HexMagic.terrainpatterns import TerrainPatterns
from HexMagic.climate import TerrainFactory

## Climate

In [ ]:


def island_demo_fixed(debug=False):
    """Create a tropical island with three volcanoes and downsampled rivers."""
    
    # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(800, 800))
    terrain , plates = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        ocean_fraction=0.6,
        debug = debug
    )
    if debug:
        print("\n=== COMPUTING CLIMATE ===")
    terrain.climate.configure(terrain,debug=debug)
    
    
    # 6. Visualize original
    if debug:
        print("\n=== RENDERING ORIGINAL ===")
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.add_climate_overlay()
    
    
    # 7. Downsample terrain (including flow)
    if debug:
        
        print("\n=== DOWNSAMPLING ===")
    smaller = terrain.downsample_climate(0.5)
    smaller.hexGrid.adjustRadius(20)
    
    # 9. Visualize downsampled version
    smaller.colorMap()
    smaller.hexGrid.update()
    smaller.add_climate_overlay()

    return smaller

isf = island_demo_fixed()
isf.colorMap
isf.hexGrid.update()
isf.hexGrid.builder.show()



Initial seeds: 25 (spacing=11)
After elimination: 21
Final unique seeds: 21


How would we profile to see what is taking time

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To profile your code and see what's taking time, you have several options. Here are the most common approaches:

**1. Simple timing with `time` module:**
```python
import time

start = time.time()
# your code here
end = time.time()
print(f"Took {end - start:.2f} seconds")
```

**2. Line-by-line profiling with `line_profiler`:**
```python
%load_ext line_profiler
%lprun -f island_demo_fixed island_demo_fixed()
```

**3. Function-level profiling with `cProfile`:**
```python
import cProfile
import pstats

cProfile.run('island_demo_fixed()', 'profile_stats')
stats = pstats.Stats('profile_stats')
stats.sort_stats('cumulative')
stats.print_stats(20)  # Show top 20 functions
```

Which approach would you like to try? The `line_profiler` is great for seeing exactly which lines are slow, while `cProfile` gives you a good overview of function calls.

In [ ]:
import cProfile
import pstats

cProfile.run('island_demo_fixed()', 'profile_stats')
stats = pstats.Stats('profile_stats')
stats.sort_stats('cumulative')
stats.print_stats(20)  # Show top 20 functions


Initial seeds: 25 (spacing=11)
After elimination: 17
Final unique seeds: 17


Tue Jan 13 16:36:09 2026    profile_stats

         6576203 function calls (6382657 primitive calls) in 4.780 seconds

   Ordered by: cumulative time
   List reduced from 396 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    4.801    4.801 {built-in method builtins.exec}
      2/1    0.003    0.001    4.801    4.801 <string>:1(<module>)
        1    0.000    0.000    4.798    4.798 /tmp/ipykernel_6810/2373736548.py:1(island_demo_fixed)
        1    0.000    0.000    4.047    4.047 /app/data/HexMagic/HexMagic/climate.py:1412(create_ocean_world)
    27648    0.220    0.000    3.824    0.000 /app/data/HexMagic/HexMagic/primitives.py:694(ring)
        1    0.016    0.016    3.769    3.769 /app/data/HexMagic/HexMagic/voronoi.py:261(generate_plate_terrain)
      238    0.092    0.000    3.466    0.015 /app/data/HexMagic/HexMagic/primitives.py:1734(outside)
   193536    0.581    0.000    2.896    0.000 /app/da

<pstats.Stats>

In [ ]:
import cProfile
import pstats

cProfile.run('island_demo_fixed()', 'profile_stats')
stats = pstats.Stats('profile_stats')
stats.sort_stats('cumulative')
stats.print_stats(20)  # Show top 20 functions

Initial seeds: 25 (spacing=11)
After elimination: 14
Final unique seeds: 14


Tue Jan 13 20:30:07 2026    profile_stats

         2308183 function calls (2308173 primitive calls) in 1.164 seconds

   Ordered by: cumulative time
   List reduced from 378 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    1.272    1.272 {built-in method builtins.exec}
      2/1    0.003    0.001    1.272    1.272 <string>:1(<module>)
        1    0.000    0.000    1.269    1.269 /tmp/ipykernel_9757/2373736548.py:1(island_demo_fixed)
        1    0.000    0.000    0.743    0.743 /app/data/HexMagic/HexMagic/climate.py:1412(create_ocean_world)
        1    0.014    0.014    0.742    0.742 /app/data/HexMagic/HexMagic/voronoi.py:261(generate_plate_terrain)
      266    0.107    0.000    0.652    0.002 /app/data/HexMagic/HexMagic/primitives.py:1806(outside)
   276680    0.309    0.000    0.618    0.000 /app/data/HexMagic/HexMagic/primitives.py:1574(hexposition_to_index)
        1    0.001    0.001    0.319

<pstats.Stats>

Any recommendations?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the profile, here are the key bottlenecks:

**Main time consumers:**
1. **`ring()` function (3.8s total)** - Called 27,648 times, this is the biggest bottleneck
2. **`rotate()` function (2.9s total)** - Called 193,536 times  
3. **`numpy.roll()` (2.1s total)** - Called by rotate, doing array operations
4. **`outside()` function (3.5s total)** - Called 238 times but expensive per call

**Recommendations:**

1. **Optimize the `ring()` function** - It's being called a lot during terrain generation. Consider:
   - Caching ring results if you're calling with the same parameters
   - Pre-computing rings if possible
   - Using vectorized operations instead of loops

2. **Reduce `rotate()` calls** - Nearly 200k calls suggest you might be:
   - Rotating unnecessarily in loops
   - Could batch rotations or avoid them entirely with different algorithms

3. **Profile with more detail on specific functions:**
```python
%load_ext line_profiler
%lprun -f Terrain.fromSeeds island_demo_fixed()
```

4. **Consider algorithmic improvements:**
   - The Voronoi generation (2.6s) might benefit from a spatial index
   - Distance-to-coast computation (0.4s) could use BFS/flood-fill instead of checking all hexes

Would you like me to look at any specific function in more detail?

The ring speed up gave us some nice speed. Anything else?

In [ ]:
isf.builder.layers = []
mountains = isf.find_peaks(7,4)
for i , epicenter in enumerate(mountains):
    isf.elevations += isf.volcano(center=epicenter, adjusted=20+ ((i+1)*30), num_rings=5)
isf.colorMap()
isf.hexGrid.update()
isf.builder.show()

## Hydrology

In [ ]:
from HexMagic.hydrology import DrainageBasins

In [ ]:
def hydrate(terrain):

    basin = DrainageBasins(terrain)

     
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )

    river_style = StyleCSS(
        "nile",
        fill = "none",
        stroke= '#23194629',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""

    mainBasins = basin.get_major(6)
    
    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        river_svg += small_river.draw()

    terrain.hexGrid.builder.adjust("rivers", river_svg)



In [ ]:
hydrate(isf)
isf.hexGrid.builder.show()